<a href="https://colab.research.google.com/github/tvaditya/intro_ds_and_ml/blob/main/Classifica%C3%A7%C3%A3o_bin%C3%A1ria_Cancer_de_mama_salvar_o_modelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto 4: Classificação binária brest cancer - classificar somente um registro e salvar o classificador

## Etapa 1: Importação das bibliotecas

In [47]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score
import torch
import torch.nn as nn
torch.__version__

'1.10.0+cu111'

## Etapa 2: Base de dados

In [48]:
np.random.seed(123)
torch.manual_seed(123)

In [49]:
previsores = pd.read_csv('entradas_breast.csv')
classe = pd.read_csv('saidas_breast.csv')

In [50]:
previsores = torch.tensor(np.array(previsores), dtype = torch.float)
classe = torch.tensor(np.array(classe), dtype = torch.float)

In [51]:
type(previsores)

torch.Tensor

## Etapa 3: Transformação dos dados para tensores


In [52]:
train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(previsores, classe), batch_size = 10, shuffle = True)

## Etapa 4: Construção do modelo

In [53]:
class classificador_torch(nn.Module):
    def __init__(self):
        super().__init__() 
        self.dense0 = nn.Linear(30, 16)
        # Mesmos parâmetors usados pelo kernel initializer do Keras [https://keras.io/initializers/, ver sessão RandomNormal]
        torch.nn.init.normal_(self.dense0.weight, mean = 0.0, std= 0.05)
        self.dense1 = nn.Linear(16, 16)
        torch.nn.init.normal_(self.dense1.weight, mean = 0.0, std= 0.05)
        self.dense2 = nn.Linear(16, 1)
        self.activation = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.output = nn.Sigmoid()
        
    def forward(self, X):
        X = self.dense0(X)
        X = self.activation(X)
        X = self.dropout(X)
        X = self.dense1(X)
        X = self.activation(X)
        X = self.dropout(X)
        X = self.dense2(X)
        X = self.output(X)
        return X

In [54]:
classificador = classificador_torch()

In [55]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(classificador.parameters(), lr = 0.001, 
                             weight_decay = 0.0001)

## Etapa 5: Treinamento do modelo

In [56]:
for epoch in range(100):
    running_loss = 0.
    
    for data in train_loader:
        inputs, labels = data
        optimizer.zero_grad()        

        outputs = classificador(inputs)
        loss = criterion(outputs, labels)
        loss.backward()    
        optimizer.step()

        running_loss += loss.item()

    print('Época %3d: perda %.5f' % (epoch+1, running_loss/len(train_loader)))

Época   1: perda 1.66508
Época   2: perda 0.64733
Época   3: perda 0.55915
Época   4: perda 0.53001
Época   5: perda 0.52182
Época   6: perda 0.50078
Época   7: perda 0.45254
Época   8: perda 0.45964
Época   9: perda 0.40986
Época  10: perda 0.44895
Época  11: perda 0.37685
Época  12: perda 0.38053
Época  13: perda 0.37975
Época  14: perda 0.37394
Época  15: perda 0.35440
Época  16: perda 0.36778
Época  17: perda 0.38368
Época  18: perda 0.34948
Época  19: perda 0.35325
Época  20: perda 0.35367
Época  21: perda 0.33867
Época  22: perda 0.31282
Época  23: perda 0.31103
Época  24: perda 0.33322
Época  25: perda 0.33423
Época  26: perda 0.34129
Época  27: perda 0.33922
Época  28: perda 0.30796
Época  29: perda 0.31137
Época  30: perda 0.32861
Época  31: perda 0.31127
Época  32: perda 0.32796
Época  33: perda 0.30930
Época  34: perda 0.33830
Época  35: perda 0.26008
Época  36: perda 0.34178
Época  37: perda 0.27992
Época  38: perda 0.31531
Época  39: perda 0.31947
Época  40: perda 0.28741


## Etapa 6: Classificar somente um registro

In [57]:
novo = torch.tensor([[15.80, 8.34, 118, 900, 0.10, 0.26, 0.08, 0.134, 0.178,
                  0.20, 0.05, 1098, 0.87, 4500, 145.2, 0.005, 0.04, 0.05, 0.015,
                  0.03, 0.007, 23.15, 16.64, 178.5, 2018, 0.14, 0.185,
                  0.84, 158, 0.363]], dtype = torch.float)

In [58]:
classificador.eval()

classificador_torch(
  (dense0): Linear(in_features=30, out_features=16, bias=True)
  (dense1): Linear(in_features=16, out_features=16, bias=True)
  (dense2): Linear(in_features=16, out_features=1, bias=True)
  (activation): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
  (output): Sigmoid()
)

In [59]:
previsao = classificador(novo)

In [60]:
previsao

tensor([[1.0000]], grad_fn=<SigmoidBackward0>)

In [61]:
previsao = previsao.detach()

In [62]:
previsao

tensor([[1.0000]])

In [63]:
# Para ter um valor que não seja um tensor
previsao = previsao.numpy()

In [64]:
type(previsao)

numpy.ndarray

In [65]:
previsao

array([[0.9999733]], dtype=float32)

In [66]:
previsao = (previsao > 0.5)
previsao

array([[ True]])

# Etapa 7: Salvar o classificador

In [67]:
# Quando salvar, você precisa chamar classificador.state_dict() (com os parêntese no final), 
# ao invés de classificador.state_dict, possui informações sobre os pesos de cada camada

In [68]:
# classificador.state_dict()

In [69]:
# Para salvar o classificador com a versão 1.5.0 do PyTorch, use o código abaixo

torch.save(classificador.state_dict(), 'checkpoint.pth')

# Load Model

In [76]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
torch.__version__

'1.10.0+cu111'

In [77]:
class classificador_torch(nn.Module):
    def __init__(self):
        super().__init__() 
        self.dense0 = nn.Linear(30, 16)
        # Mesmos parâmetors usados pelo kernel initializer do Keras [https://keras.io/initializers/, ver sessão RandomNormal]
        torch.nn.init.normal_(self.dense0.weight, mean = 0.0, std= 0.05)
        self.dense1 = nn.Linear(16, 16)
        torch.nn.init.normal_(self.dense1.weight, mean = 0.0, std= 0.05)
        self.dense2 = nn.Linear(16, 1)
        self.activation = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.output = nn.Sigmoid()
        
    def forward(self, X):
        X = self.dense0(X)
        X = self.activation(X)
        X = self.dropout(X)
        X = self.dense1(X)
        X = self.activation(X)
        X = self.dropout(X)
        X = self.dense2(X)
        X = self.output(X)
        return X

In [78]:
classificador = classificador_torch()

In [79]:
state_dict = torch.load('/content/checkpoint.pth')

In [80]:
state_dict

OrderedDict([('dense0.weight',
              tensor([[ 6.0988e-02,  2.7366e-01,  2.6954e-01,  1.0958e-02, -1.6898e-01,
                       -6.4578e-02, -1.4541e-01,  1.7019e-01,  2.8645e-01,  3.0500e-01,
                        2.8659e-02, -9.6251e-02, -2.7989e-02,  4.0410e-02,  3.5745e-03,
                       -3.3540e-02,  1.2066e-01,  2.7935e-01,  1.1087e-01,  2.6153e-03,
                        1.0095e-01,  2.3742e-01,  1.0516e-01, -6.4800e-02, -1.0049e-01,
                       -1.8869e-02, -3.3261e-02, -6.7347e-02,  1.3307e-01, -5.1373e-02],
                      [ 1.1007e-01,  1.7923e-01,  3.9027e-01,  5.6854e-02, -1.7357e-01,
                       -2.0041e-01,  1.4577e-02,  8.4380e-02,  2.6528e-02,  2.8121e-01,
                       -1.9251e-01, -9.9850e-02,  1.2540e-03, -3.3471e-01,  2.6606e-03,
                       -3.8310e-02,  7.9879e-02,  1.9506e-01,  5.8244e-01, -8.0416e-04,
                        1.9718e-01,  3.3930e-02,  1.9907e-01, -1.1949e-01,  7.6358e-02,


In [81]:
classificador.load_state_dict(state_dict)

<All keys matched successfully>

In [83]:
novo = torch.tensor([[15.80, 8.34, 118, 900, 0.10, 0.26, 0.08, 0.134, 0.178,
                      0.20, 0.05, 1098, 0.87, 4500, 145.2, 0.005, 0.04, 0.05, 0.015,
                      0.03, 0.007, 23.15, 16.64, 178.5, 2018, 0.14, 0.185,
                      0.84, 158, 0.363]])

In [84]:
classificador.eval()
previsao = classificador(novo)
previsao = (previsao.detach().numpy() > 0.5)
previsao

array([[ True]])

In [85]:
previsores = pd.read_csv('entradas_breast.csv')
classe = pd.read_csv('saidas_breast.csv')

In [86]:
previsores = torch.tensor(np.array(previsores), dtype = torch.float)
classe = torch.tensor(np.array(classe), dtype = torch.float)

In [87]:
previsoes = classificador.forward(previsores).detach()

In [88]:
previsoes

tensor([[5.7493e-09],
        [1.9278e-05],
        [5.0140e-04],
        [8.3125e-05],
        [5.4147e-03],
        [3.2701e-01],
        [6.8382e-06],
        [7.6171e-02],
        [1.3481e-01],
        [3.9855e-05],
        [3.1114e-02],
        [1.2291e-03],
        [9.7506e-02],
        [6.5839e-01],
        [6.5839e-01],
        [1.9841e-01],
        [5.7210e-03],
        [3.0082e-04],
        [1.3265e-10],
        [9.5545e-01],
        [6.5839e-01],
        [1.0000e+00],
        [2.1743e-01],
        [7.4527e-11],
        [4.4244e-11],
        [3.3947e-05],
        [3.1030e-01],
        [9.0055e-03],
        [5.4735e-05],
        [1.2439e-01],
        [1.3931e-04],
        [1.0066e-02],
        [2.9152e-03],
        [2.1897e-06],
        [8.5700e-04],
        [3.3864e-02],
        [6.2787e-01],
        [9.9993e-01],
        [6.5839e-01],
        [2.4901e-01],
        [4.7507e-01],
        [9.5795e-01],
        [4.9318e-05],
        [4.3626e-02],
        [6.6540e-01],
        [1

In [89]:
F.binary_cross_entropy(previsoes, classe).numpy()

array(0.16279869, dtype=float32)

In [90]:
accuracy_score(classe.numpy(), (previsoes > 0.5).numpy())

0.9402460456942003